## OpenMMTools Integration

This enables OPS to use any testsystem from openmmtools to be used as well as add ANY possible OpenMM setup, not just the ones that we provided before. The old OpenMMEngine has been rename intp `SimpleOpenMMEngine`

In [1]:
import openpathsampling as paths
import openmmtools as omt
import simtk.openmm as omm
import simtk.unit as u
import mdtraj as md

import openpathsampling.engines.openmm as eng

from __future__ import print_function

Create Alanine in Vacuum and run it using OPS.

In [2]:
testsystem = omt.testsystems.AlanineDipeptideVacuum()

Let's have a look at the content

In [3]:
#! skip
{ key: type(value) for key, value in testsystem.__dict__.items()}

{'_positions': simtk.unit.quantity.Quantity,
 '_system': simtk.openmm.openmm.System,
 '_topology': simtk.openmm.app.topology.Topology}

An `OpenMM` simulation in OPS needs 3 ingredients to function
1. A template snapshot (which contains topolgy and coordinates/velocities), 
2. an OpenMM system object, and 
3. an OpenMM integrator object.

In the following we will get these parts.

#### 1. The template

The information for the template is contained in the `_topology` and the `_positions` object.

In [4]:
template = eng.snapshot_from_testsystem(testsystem)

#### 2. The system

This is easy since we directly get it from the testsytem objects

In [5]:
system = testsystem.system

#### 3. The integrator

The openmmtools system is (almost) independent of the integrator and `openmm` provides us with lots of options. For now we pick something simple.

In [6]:
integrator = omm.VerletIntegrator(
    0.002 * u.picoseconds    
)

build the engine

In [7]:
engine = eng.Engine(
    template.topology, 
    system, 
    integrator)

Let's run a simulation of 10 steps.

In [8]:
traj = engine.generate(template, [paths.LengthEnsemble(10).can_append])

In [9]:
print(traj)

Trajectory[10]


In [10]:
#! skip
print traj[5].coordinates

[[ 0.18766727  0.11599673  0.01245007]
 [ 0.19458343  0.22444926  0.00401418]
 [ 0.14436732  0.27122673  0.0886971 ]
 [ 0.14886758  0.25747463 -0.08926168]
 [ 0.33994529  0.26638272  0.0060984 ]
 [ 0.4270958   0.18970114  0.04380052]
 [ 0.36886489  0.38998333 -0.03181564]
 [ 0.29336685  0.45243087 -0.05633813]
 [ 0.49859309  0.45657608 -0.02400124]
 [ 0.55814373  0.41515255  0.0573551 ]
 [ 0.57325345  0.43391329 -0.15561195]
 [ 0.51911265  0.48312771 -0.23640625]
 [ 0.67427611  0.47320506 -0.14414112]
 [ 0.5842306   0.32728142 -0.17536473]
 [ 0.47495213  0.60669041 -0.00127664]
 [ 0.35958388  0.64924192  0.00474685]
 [ 0.58359265  0.68379176  0.00902626]
 [ 0.6747424   0.6406613   0.00332439]
 [ 0.57127422  0.82897234  0.01853088]
 [ 0.46718922  0.86095428  0.02347775]
 [ 0.62214291  0.86301541  0.10872208]
 [ 0.61715877  0.87434274 -0.06931645]] nm


In [11]:
psi = md.compute_psi(traj.to_mdtraj())

/Users/jan-hendrikprinz/anaconda/lib/python2.7/site-packages/mdtraj/utils/validation.py:116: TypeCastPerformanceWarning: Casting unitcell_vectors dtype=float64 to <type 'numpy.float32'> 
  TypeCastPerformanceWarning)


In [12]:
#! ignore
psi[1][3:8]

array([[ 3.10614586],
       [-3.09586358],
       [ 3.1367557 ],
       [ 3.12832689],
       [ 3.0931778 ]], dtype=float32)

In [13]:
st = paths.Storage('engine_store_test.nc', mode='w')

In [14]:
st.engines.save(engine);
st.tag['template'] = template

The engine is stored by using the XML serialization of OpenMM and put this into the storage.

In [15]:
#! skip
st.variables['engines_json'][0][0:256] + '...'

u'{"_cls":"OpenMMEngine","_dict":{"integrator_xml":"<?xml version=\\"1.0\\" ?>\\n<Integrator constraintTolerance=\\"1e-05\\" stepSize=\\".002\\" type=\\"VerletIntegrator\\" version=\\"1\\"\\/>\\n","system_xml":"<?xml version=\\"1.0\\" ?>\\n<System openmmVersion=\\"7.1\\" type...'

In [16]:
st.save(traj);

In [17]:
st.close()

### try to load the stored engine

In [18]:
st = paths.AnalysisStorage('engine_store_test.nc')

In [19]:
engine = st.engines[0]

Run another 10 steps to test

In [20]:
template = st.tag['template']

In [21]:
traj = engine.generate(template, [paths.LengthEnsemble(10).can_append])

In [22]:
print(traj)

Trajectory[10]


In [23]:
st.close()

And save the trajectory for future use (other tests)

In [24]:
st = paths.Storage('engine_store_test.nc', "a")
st.save(traj)
st.close()